In [22]:
import pandas as pd
import os
import plotly.graph_objects as go

import warnings
warnings.filterwarnings(
    "ignore",
    message="The behavior of DatetimeProperties.to_pydatetime is deprecated",
    category=FutureWarning
)


# Create color palette
color_palette = ['rgba(0, 74, 174, 1)', 'rgba(231, 17, 94, 1)', 'rgba(255, 195, 27, 1)',  'rgba(130, 54, 199, 1)',
                 'rgba(1, 137, 67, 1)', 'rgba(252, 147, 19, 1)', 'rgba(159, 209, 255, 1)', 'rgba(119, 7, 71, 1)']

DATA_FOLDER = "/Users/akimovh/Library/CloudStorage/GoogleDrive-akimovhresearch@gmail.com/My Drive/predatory_pricing_airlines/data/"
PLOT_FOLDER = "/Users/akimovh/Library/CloudStorage/GoogleDrive-akimovhresearch@gmail.com/My Drive/predatory_pricing_airlines/plots/"

In [23]:
folder_path = DATA_FOLDER + "T-F41/raw"

# List all files in the folder
files = [f for f in os.listdir(folder_path) if f.endswith(('.csv', '.xlsx', '.json'))]  # Modify for other formats

# Initialize an empty list to store DataFrames
df_list = []

# Loop through the files and read them
for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    df_list.append(df)

# Concatenate all DataFrames into one
costs = pd.concat(df_list, ignore_index=True)
costs = costs.query("UNIQUE_CARRIER == 'HA'")
costs = costs.query("REGION == 'D'")
costs['date'] = pd.to_datetime(costs['YEAR'].astype(str) + '-' + ((costs['QUARTER'] - 1) * 3 + 1).astype(str) + '-01')
costs = costs.sort_values(by = 'date')

In [24]:
# -----------------------
# Prepare data
# -----------------------
plot_data = costs[['date', 'INCOME_PRE_TAX']].copy()
plot_data['INCOME_PRE_TAX'] *= 1000

year_tick_vals = (
    plot_data
    .groupby(plot_data['date'].dt.year)['date']
    .first()
    .to_numpy()
)

unique_years = plot_data['date'].dt.year.unique()

# -----------------------
# Create figure
# -----------------------
fig = go.Figure()

# Income line with markers
fig.add_trace(
    go.Scatter(
        x=plot_data['date'],
        y=plot_data['INCOME_PRE_TAX'],
        mode='lines+markers',
        line=dict(
            color='rgba(0, 74, 174, 1)',
            width=3
        ),
        marker=dict(
            color='rgba(0, 74, 174, 1)',
            size=10,
            symbol='circle'
        )
    )
)

# Horizontal zero line
fig.add_shape(
    type='line',
    x0=plot_data['date'].min(), y0=0,
    x1=plot_data['date'].max(), y1=0,
    line=dict(color='black', width=1)
)

# Southwest entry vertical line
fig.add_shape(
    type='line',
    x0=pd.to_datetime('2019-04-01'), y0=plot_data['INCOME_PRE_TAX'].min(),
    x1=pd.to_datetime('2019-04-01'), y1=plot_data['INCOME_PRE_TAX'].max(),
    line=dict(color='rgba(231, 17, 94, 1)', width=1.5, dash='dash')
)

fig.add_annotation(
    x=pd.to_datetime('2019-04-01'),
    y=plot_data['INCOME_PRE_TAX'].max() + 10_000_000,
    text='SW entry',
    showarrow=False,
    font=dict(
        family='Arial',
        size=20,
        color='rgba(231, 17, 94, 1)'
    )
)

# COVID-19 shaded period
fig.add_shape(
    type='rect',
    x0='2020-02-01',
    x1='2021-04-01',
    y0=-200_000_000,
    y1=180_000_000,
    fillcolor='lightgray',
    opacity=0.3,
    layer='below',
    line_width=0
)

fig.add_annotation(
    x='2020-08-10',
    y=130_000_000,
    text='COVID-19',
    showarrow=False,
    align='center',
    font=dict(
        family='Arial',
        size=20,
        color='rgba(231, 17, 94, 1)'
    ),
    bgcolor='white',
    opacity=0.7,
    borderpad=4
)

# -----------------------
# Layout
# -----------------------
fig.update_layout(
    width=1280,
    height=720,
    font=dict(
        family='Arial',
        size=20
    ),
    xaxis=dict(
        title='Date (Quarter)',
        title_font=dict(size=22),
        tickmode='array',
        tickvals=year_tick_vals,
        ticktext=unique_years,
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    yaxis=dict(
        title='Pre-tax Net Income (Millions US Dollars)',
        title_font=dict(size=22),
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True,
        tickformat=','
    ),
    margin=dict(t=50, b=80, l=100, r=40),
    template='plotly_white',
    plot_bgcolor='rgba(255, 255, 255, 1)'
)

fig.show()

fig.write_image(
    PLOT_FOLDER + "HA_income.png",
    scale=3
)


# Price-cost analysis